Vandita Shukla, Student Number-17141590

# Practical 7 - Part 2A
This second half of the lab explores the geometry of a single camera. In 2B the goal is to use a set of correspondance points to estimate a transformation matrix from a plane's 3D space to camera space and use that matrix to project some other points into camera space.

I'll work on building two components that we need for 2B, a method to estimate that transformation and a method that can project points into camera image space.

First tackle the projection method, `projectiveCamera`. We want to find the image space coordinates, `XImCart`, of a set of 3D world coordinates, `XCart`, given a camera intrinsics matrix `K` and an extrinsics matrix `T`.

The second component is a method to estimate a Eucledian transformation, `TEst`, that takes us from a plane's 3D coordinate space to 3D camera space by utilizing a given set of points in camera image space, `XImCart`, and a set of corresponding points in world space, `XCart`. Essentially we want to compute the extrinsics matrix we can use in `projectiveCamera`.

Estimating the camera pose will involve calculating a homography, so as a shortcut I will copy over my functions from part 1A/1B 

## Import libraries 

In [1]:
%matplotlib inline
import os 
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

In [2]:
def projectiveCamera(K,T,XCart):
    # The goal of this function is to project points in XCart through projective camera
    # defined by intrinsic matrix K and extrinsic matrix T. In essence, this function takes a set of points 
    # in 3D world space, XCart, and projects them into camera image space by applying the extrinsic matrix T 
    # and then applying the intrinsic matrix K.
    # 
    # There are three steps.
    # 1) Move from world space to camera space. 
    #            camera space points = extrinsics T * world space points 
    #
    # 2) Applying the intrinsics matrix to the camera space points after normalizing
    #           homogeneous image space points = K * normalized camera space points
    # 
    # 3) Move to image space cartesian points from image space homogeneous points, involves a 
    # normalization using the third row.
    
    
    
    XHom = np.concatenate((XCart, np.ones((1,XCart.shape[1]))), axis=0)
    
    XHom = np.matmul(T,XHom)
    #print('XHOM',XHom)
    XCamHom = XHom[0:(XHom.shape[0]-1),:]
    #print('XCamHom',XCamHom)
    xImHom = np.matmul(K,XCamHom)
    xImCart = xImHom[0:2,:] / np.tile([xImHom[2,:]],(2,1))
    return xImCart


## Camera Projection

First we'll write up the function that can take us from 3D world space, `XCart`, to camera image space, `XImCart`, using an extrinsics matrix `T` and an intrinsics matrix `K` that are provided. The previous block houses this function.

The result here is the cartesian image space point coordinates, `XImCart`, of the 3D points `XCart`. If `XCart` represents a box in the world then we now know where the box's vertices would land in image space.

To verify that your solution is correct please compare your image space points to those in the comment.

Once they match, move on to the next bit - estimating a transformation! 

In [3]:
# We assume that the camera intrinsics matrix K is known and has values:
K = np.array([[640, 0, 320],
             [0, 640, 240],
             [0, 0, 1]])

# We will assume an object co-ordinate system with the Z-axis pointing upwards and the origin
# in the centre of the plane. There are five known points on the plane with coordinates (mm):
XCart = np.array([[-100, -100,  100,  100, 0],
                  [-100,  100,  100, -100, 0],
                  [   0,    0,    0,    0, 0]])

# We assume the correct transformation from the plane co-ordinate system to the
# camera co-ordinate system (extrinsic matrix) is:
T = np.array([[0.9851,  -0.0492,  0.1619,  46.00],
             [-0.1623,  -0.5520,  0.8181,  70.00],
             [0.0490,  -0.8324, -0.5518,  500.89],
             [0,        0,       0,       1]])
# T houses a rotation matrix and a translation matrix. The last row is for homogeneous point calculation.


# Use the general pin-hole projective camera model discussed in the lectures to estimate 
# where the four points on the plane will appear in the image.  Fill in the
# details of the function "projectiveCamera"
XImCart = projectiveCamera(K,T,XCart)


# TA said that it should be around:
# [267.4170882  230.95045427 531.42492013 482.36049098 378.77537982]
# [396.26814909 288.11435494 237.83410247 358.39940241 329.44079538]

### You've implemented both of these functions in 1A and 1B already, so feel free to copy them in here. You'll need them for this next part.

In [4]:
def solveAXEqualsZero(A):
    [U,L,Vh]=np.linalg.svd(A,full_matrices=True)
    V=np.transpose(Vh)
    [x, y]=np.shape(V)
    h=np.zeros([x,1])
    h=V[:,y-1]
    return h

In [5]:
def calcBestHomography(pts1Cart, pts2Cart):
    
    # This function should apply the direct linear transform (DLT) algorithm to calculate the best 
    # homography that maps the cartesian points in pts1Cart to their corresonding matching cartesian poitns 
    # in pts2Cart.
    
    # This function calls solveAXEqualsZero. Make sure you are wary of how to reshape h into a 3 by 3 matrix. 

    n_points = pts1Cart.shape[1]
    
    H = np.identity(3)
    # First convert points into homogeneous representation
    # Hint: we've done this before  in the skeleton code we provide.
    pts1Hom = np.concatenate((pts1Cart, np.ones((1,pts1Cart.shape[1]))), axis=0)
    pts2Hom = np.concatenate((pts2Cart, np.ones((1,pts2Cart.shape[1]))), axis=0)
    # Then construct the matrix A, size (n_points * 2, 9)
    # 
    A = np.zeros([n_points*2,9])
    
    for i in range(n_points):
               
        A[i*2,:]=[0,0,0,-pts2Hom[0,i],-pts2Hom[1,i],-1,pts1Hom[1,i]*pts2Hom[0,i],pts1Hom[1,i]*pts2Hom[1,i],pts1Hom[1,i]]
        A[((i*2)+1),:]=[pts2Hom[0,i],pts2Hom[1,i],1,0,0,0,-pts1Hom[0,i]*pts2Hom[0,i],-pts1Hom[0,i]*pts2Hom[1,i],-pts1Hom[0,i]]
    # Solve Ah = 0 using solveAXEqualsZero and get h.
    
    h = solveAXEqualsZero(A)
    # Reshape h into the matrix H, values of h go first into rows of H
    H = np.reshape(h,(3,3))
    #H[0,:]=[h[0,],h[1,],h[2,]]
    #H[1,:]=[h[3,],h[4,],h[5,]]
    #H[2,:]=[h[6,],h[7,],h[8,]]
    return H

In [6]:
# Read the next cell first for context!

def estimatePlanePose(XImCart,XCart,K):
    # The goal of this function is to estimate the pose of a plane relative to camera (extrinsic matrix)
    # given points in image space xImCart, points in 3D world space XCart, and an intrinsics matrix K.
    
    XImHom = np.concatenate((XImCart, np.ones((1,XImCart.shape[1]))), axis=0)
    
    XCamHom = np.matmul(np.linalg.inv(K),XImHom)
    XCamHom = XCamHom/XCamHom[2,:]
    #print(XCamHom)
    
    # Estimate homography H mapping homogeneous (x,y) coordinates of positions
    # in real world to XCamHom (convert XCamHom to Cartesian, calculate the homography) -
    # use the routine you wrote for Practical 1B
    XCamCart = XCamHom[0:(XCamHom.shape[0]-1),:]
    
    #print('XCART',XCart)
    XCartuv = XCart[0:(XCart.shape[0]-1),:]
    #print('XCARTuv',XCartuv)
    H = calcBestHomography(XCamCart,XCartuv) 
    #print ('Homog',H)
    # Estimate first two columns of rotation matrix R from the first two
    # columns of H using the SVD. NOTE: You do not need to transpose v from linalg.svd    
    Hx = H[:,0:H.shape[1]-1]
    [U,L,V] = np.linalg.svd(Hx)
    
    R12 = np.zeros((3,2));
    R12 = np.matmul(U,[[1, 0],[0, 1],[0, 0]])
    R12 = np.matmul(R12,V)
    # Estimate the third column of the rotation matrix by taking the cross
    # product of the first two columns
    R3 = np.cross(R12[:,0],R12[:,1])
    R3 = np.reshape(R3,(3,1))
    # Check that the determinant of the rotation matrix is positive - if
    # not then multiply last column by -1.
    R = np.concatenate((R12,R3),axis=1)
    
    det = np.linalg.det(R)
    
    if det<0:
        R[:,-1]=R[:,-1]*(-1)
    # Estimate the translation t by finding the appropriate scaling factor k
    # and applying it to the third colulmn of H
    sum = 0;
    for i in range(3):
        for j in range(2):
            sum += H[i,j]/R[i,j]
    scalefactor = sum/6
    
    Tr = np.reshape((H[:,-1]/scalefactor),(3,1))
    # Check whether t_z is negative - if it is then multiply t by -1 and
    # the first two columns of R by -1.
    
    if Tr[2,:]<0:
        Tr=-Tr
        R[:,0]=-R[:,0]
        R[:,1]=-R[:,1]
        
    T = np.concatenate((R,Tr),axis=1) 
    T = np.concatenate((T,[[0, 0, 0, 1]]),axis=0)
    # Assemble transformation into matrix form

    
    return T 

## Now the juicy bit: Estimating an Extrinsics Matrix, T

The problem: We are given an instrinsics matrix `K`, a set of 3D world points `XCart`, and a set of corresponding image space coordinates in `XImCart`. `K` and `XCart` have already been defined a few cells back and you've calculated `XImCart` by virtue of the exercise you've completed with camera projection. What we don't have is an extrinsics matrix, `T`. We need to estimate this and you'll need to fill in `estimatePlanePose` and return `TEst`.

Again you can start by negating the noise we add to XImCart to make sure you're on the right track.

In [7]:
# TA Suggestion: Add noise (standard deviation of one pixel in each direction) to the pixel positions
# to simulate having to find these points in a noisy image. Store the results back in xImCart
XImCartNoisy = XImCart #add noise here

# Now we will take the image points and the known positions on the card and estimate  
# the extrinsic matrix using the algorithm discussed in the lecture.  Fill in the details of 
# the function estimatePlanePose
TEst = estimatePlanePose(XImCartNoisy,XCart,K)

# If you have got this correct, TEst should closely resemble the groundtruth, T.
np.set_printoptions(precision=3)
print(TEst)
print("\n")
print(T)


[[ 9.855e-01 -4.936e-02  1.622e-01  4.605e+01]
 [-1.623e-01 -5.520e-01  8.179e-01  7.008e+01]
 [ 4.916e-02 -8.324e-01 -5.520e-01  5.015e+02]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00]]


[[ 9.851e-01 -4.920e-02  1.619e-01  4.600e+01]
 [-1.623e-01 -5.520e-01  8.181e-01  7.000e+01]
 [ 4.900e-02 -8.324e-01 -5.518e-01  5.009e+02]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00]]
